In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [7]:
train = pd.read_csv('../input/spooky-author-identification/train.csv')
test = pd.read_csv('../input/spooky-author-identification/test.csv')


In [8]:
train.head()

In [9]:
test.head()

In [10]:
plt.figure(figsize = (12,6))
train['author'].value_counts().plot(kind = 'bar')

## Distribution is almost balanced among three of them .
## lets do some feature engg 
 This consists of two main parts.

1.  Meta features - features that are extracted from the text like number of words, number of stop words, number of punctuations etc
2. Text based features - features directly based on the text / words like frequency, svd, word2vec etc.


## meta features 

1. Number of words in the text
2.  Number of unique words in the text
3. Number of characters in the text
4. Number of stopwords
5. Number of punctuations
6. Number of upper case words
7. Number of title case words
8. Average length of the words

In [11]:
stop_words = set(stopwords.words("english"))
stop_words
len(stop_words)

In [12]:
## number of words in text 
train['num_words'] = train['text'].apply(lambda x : len(str(x).split()))
test['num_words'] = test['text'].apply(lambda x : len(str(x).split()))

## number of unique words in the text 
train['unique_words'] = train['text'].apply(lambda x : len(set(str(x).split())))
test['unique_words'] = test['text'].apply(lambda x : len(set(str(x).split())))

## num of characters in the text
train['char'] = train['text'].apply(lambda x : len(str(x)))
test['char'] = test['text'].apply(lambda x : len(str(x)))

## num of stop_words
train['stop_words'] = train['text'].apply(lambda x : len([w for w in str(x).lower().split() if w in stop_words]))
test['stop_words'] = test['text'].apply(lambda x : len([w for w in str(x).lower().split() if w in stop_words]))

## num of punctuations 
train['punc'] = train['text'].apply(lambda x : len([c for c in str(x) if c in string.punctuation]))
test['punc'] = test['text'].apply(lambda x : len([c for c in str(x) if c in string.punctuation]))

## num of upper case words
train['upper_case'] = train['text'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test['upper_case'] = test['text'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## number of title case words 
train['title_case'] = train['text'].apply(lambda x : len([c for c in str(x).split() if c.istitle()]))
test['title_case'] = test['text'].apply(lambda x : len([c for c in str(x).split() if c.istitle()]))

## average len words 
train['mean_words'] = train['text'].apply(lambda x : np.mean([len(w) for w in str(x).split()]))
test['mean_words'] = test['text'].apply(lambda x : np.mean([len(w) for w in str(x).split()]))

In [13]:
## plot some graphs based on some new features 
plt.figure(figsize = (8,6))
sns.violinplot(x = 'author' , y = 'num_words' , data = train)

In [14]:
train['num_words'].loc[train['num_words']>100] = 100
plt.figure(figsize = (12,8))
sns.violinplot(x = 'author', y = 'num_words', data = train)

In [15]:
from statistics import mode
mode(train['num_words'])

In [16]:
eap = train[train['author'] == 'EAP']['text'].values
eap

In [17]:
eap_64 = b'...'
## This is the picture of a ferrari car 

In [18]:
## plot the magic of word cloud , visualisation just for fun 
## lets build a great visualiation of word cloud 
import codecs

import base64
from scipy.misc import imread
import imageio
# Generate the Mask for EAP
f1 = open("eap.png", "wb")
f1.write(codecs.decode(eap_64,'base64'))
f1.close()
img1 = imread("eap.png")
# img = img.resize((980,1080))
hcmask = img1






In [19]:
from wordcloud import WordCloud, STOPWORDS
plt.figure(figsize = (16,13))
wc = WordCloud(background_color="black", max_words=10000, 
               mask=hcmask, stopwords=STOPWORDS, max_font_size= 40)
wc.generate(''.join(eap))
plt.imshow(wc.recolor( colormap= 'PuBu' , random_state=17), alpha=0.9)
